In [7]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = gdf_edges.index
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

osmnx version 2.0.1


### Download network in a boundingbox

In [16]:
min_lon, max_lon = 12.38, 12.62
min_lat, max_lat = 41.8, 41.95

boundary_polygon = Polygon([(min_lon, min_lat), 
                            (max_lon, min_lat), 
                            (max_lon, max_lat), 
                            (min_lon, max_lat)])

G = ox.graph_from_polygon(boundary_polygon, 
                          custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link|living_street|service|busway"]'
                          )

gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
print(f"Number of nodes: {len(gdf_nodes)}")
print(f"Number of edges: {len(gdf_edges)}")


Number of nodes: 50216
Number of edges: 103787


### Download network in a city

In [ ]:
import osmnx as ox
import geopandas as gpd
import os
from fmm import Network

place = "Rome, Italy"

G = ox.graph_from_place(
    place,
    custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|'
    'residential|motorway_link|trunk_link|primary_link|secondary_link|tertiary_link|living_street|'
    'service|busway"]'
)

gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
output_folder = "rome"
os.makedirs(output_folder, exist_ok=True)

# Save nodes and edges as shapefiles
gdf_nodes.to_file(os.path.join(output_folder, "nodes.shp"))
gdf_edges.to_file(os.path.join(output_folder, "edges.shp"))
edges = gpd.read_file(os.path.join(output_folder, "edges.shp"))

# Add a unique 'fid' column
edges["fid"] = edges.index  # Unique index as ID
edges.to_file(os.path.join(output_folder, "edges_fixed.shp"), encoding="utf-8")

# Load the edges shapefile
edges = gpd.read_file("../osmnx_example/rome/edges.shp")
edges["fid"] = edges.index  # Unique index as ID
edges.to_file("../osmnx_example/rome/edges.shp", encoding="utf-8")

print("Fixed shapefile saved.")
network = Network("../osmnx_example/rome/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))

/tmp/ipykernel_32437/3722678264.py:23: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(os.path.join(output_folder, "nodes.shp"))
/home/dieter/thesis/.venv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(


Fixed shapefile saved.
[2025-03-13 15:37:26.253] [info] [network.cpp:72] Read network from file ../osmnx_example/rome/edges.shp
[2025-03-13 15:37:26.864] [info] [network.cpp:172] Number of edges 188608 nodes 88690
[2025-03-13 15:37:26.864] [info] [network.cpp:173] Field index: id 20 source 0 target 1
[2025-03-13 15:37:26.950] [info] [network.cpp:176] Read network done.
Nodes 88690 edges 188608


### Download network with a boundary

In [ ]:
# Download by a boundary polygon in geojson
import osmnx as ox
from shapely.geometry import shape
json_file = open("stockholm_boundary.geojson")
import json
data = json.load(json_file)
boundary_polygon = shape(data["features"][0]['geometry'])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
save_graph_shapefile_directional(G, filepath='stockholm')